In [18]:
# -*- coding: utf-8 -*-
"""UAS_Penalaran_Komputer_Tahap_2_Final_Corrected.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1I4cFvpXiUeHAhi8k4w0_RV4gkqXEdgeK
"""

# Instalasi Library
print("Menginstal library yang dibutuhkan...")
# Output instalasi diarahkan ke /dev/null agar tidak terlalu banyak log.
!pip install pandas requests beautifulsoup4 pdfminer.six lxml > /dev/null 2>&1
print("Instalasi selesai.")

# Koneksi Google Drive
print("Menghubungkan Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive terhubung. Data akan disimpan di '/content/drive/MyDrive/CBR_Data'.")

Menginstal library yang dibutuhkan...
Instalasi selesai.
Menghubungkan Google Drive...
Mounted at /content/drive
Google Drive terhubung. Data akan disimpan di '/content/drive/MyDrive/CBR_Data'.


In [19]:
import pandas as pd
import os
import re
from datetime import date
from typing import Dict, Any, List, Union, Tuple

def create_path(folder_name: str) -> str:
    """
    Membuat folder di Google Drive jika belum ada.
    Args:
        folder_name (str): Nama folder yang akan dibuat (e.g., "CBR_Data/Raw_Texts").
    Returns:
        str: Jalur lengkap ke folder yang dibuat.
    """
    path = os.path.join('/content/drive/MyDrive/', folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Folder '{path}' berhasil dibuat di Google Drive.")
    else:
        print(f"Folder '{path}' sudah ada di Google Drive.")
    return path

def load_scraped_data(processed_data_path: str, keyword_for_filename_hint: str = None) -> pd.DataFrame:
    """
    Memuat data metadata CSV yang sudah di-scrape dari Tahap 1.
    Args:
        processed_data_path (str): Jalur folder tempat CSV metadata disimpan.
        keyword_for_filename_hint (str, optional): Petunjuk keyword/hash yang mungkin digunakan
                                                   untuk mempersempit pencarian file CSV.
                                                   Jika None, akan mencari file CSV terbaru.
    Returns:
        pd.DataFrame: DataFrame yang berisi data putusan.
    """
    print(f"\n--- Memulai load_scraped_data ---")
    print(f"Jalur folder yang diperiksa: {processed_data_path}")
    print(f"Keyword/Hint yang diberikan: '{keyword_for_filename_hint}'")

    if not os.path.exists(processed_data_path):
        print(f"DIAGNOSIS: ERROR FATAL - Folder '{processed_data_path}' TIDAK DITEMUKAN. Pastikan Tahap 1 berhasil membuat folder ini dan Drive terhubung dengan benar.")
        return pd.DataFrame()

    all_files_in_dir = os.listdir(processed_data_path)
    print(f"File-file yang ditemukan di '{processed_data_path}': {all_files_in_dir}")

    csv_filename_to_load = None

    # Opsi 1: Mencoba nama file yang sangat spesifik (berdasarkan hint dan tanggal hari ini)
    today_str = date.today().strftime("%Y-%m-%d")
    if keyword_for_filename_hint:
        specific_filename_today = f"putusan_ma_{keyword_for_filename_hint}_{today_str}.csv"
        full_specific_path_today = os.path.join(processed_data_path, specific_filename_today)
        if os.path.exists(full_specific_path_today):
            csv_filename_to_load = specific_filename_today
            print(f"DIAGNOSIS: Ditemukan file CSV spesifik hari ini: {csv_filename_to_load}")
        else:
            print(f"DIAGNOSIS: File CSV spesifik hari ini '{specific_filename_today}' tidak ditemukan.")

    # Opsi 2: Mencari file CSV yang cocok dengan hint atau yang terbaru di folder
    if not csv_filename_to_load:
        matching_files_prefix = [f for f in all_files_in_dir if f.startswith('putusan_ma_') and f.endswith('.csv')]

        if keyword_for_filename_hint:
            filtered_files = [f for f in matching_files_prefix if keyword_for_filename_hint in f]
            if filtered_files:
                csv_filename_to_load = max(filtered_files, key=lambda f: os.path.getmtime(os.path.join(processed_data_path, f)))
                print(f"DIAGNOSIS: Ditemukan file CSV yang cocok dengan hint dari daftar: {csv_filename_to_load}")
            elif matching_files_prefix:
                csv_filename_to_load = max(matching_files_prefix, key=lambda f: os.path.getmtime(os.path.join(processed_data_path, f)))
                print(f"DIAGNOSIS: Tidak ada file yang cocok dengan hint, memuat file CSV terbaru secara umum: {csv_filename_to_load}")
        elif matching_files_prefix:
            csv_filename_to_load = max(matching_files_prefix, key=lambda f: os.path.getmtime(os.path.join(processed_data_path, f)))
            print(f"DIAGNOSIS: Memuat file CSV terbaru yang ditemukan di folder: {csv_filename_to_load}")
        else:
            print(f"DIAGNOSIS: Tidak ada file CSV 'putusan_ma_*.csv' ditemukan di folder '{processed_data_path}'.")
            return pd.DataFrame() # Mengembalikan DataFrame kosong jika tidak ada file


    if csv_filename_to_load:
        full_csv_path = os.path.join(processed_data_path, csv_filename_to_load)
        print(f"DIAGNOSIS: Akan mencoba memuat file: {full_csv_path}")
        try:
            df = pd.read_csv(full_csv_path)
            print(f"Berhasil memuat {len(df)} dokumen dari {full_csv_path}")
            return df
        except pd.errors.EmptyDataError:
            print(f"DIAGNOSIS: ERROR - File CSV '{full_csv_path}' kosong. Tidak ada data untuk diproses.")
            return pd.DataFrame()
        except Exception as e:
            print(f"DIAGNOSIS: ERROR - Gagal saat memuat CSV '{full_csv_path}': {e}")
            print(f"Ini mungkin disebabkan oleh file rusak, format tidak valid, atau masalah izin baca.")
            return pd.DataFrame()
    else:
        print("DIAGNOSIS: Tidak ada file CSV yang cocok atau ditemukan untuk dimuat.")
        return pd.DataFrame()

def summarize_facts(text: str) -> str:
    """
    Mengekstrak ringkasan fakta dari teks putusan.
    Ini adalah implementasi SANGAT SEDERHANA. Anda perlu mengembangkan ini.
    """
    if not isinstance(text, str) or not text:
        return ""
    sentences = re.split(r'(?<=[.!?])\s+', text)
    summary = " ".join(sentences[:3]) if len(sentences) > 0 else text[:200]
    return summary.strip() + "..." if len(summary) > 200 else summary.strip()

def extract_legal_arguments(text: str) -> str:
    """
    Mengekstrak argumen hukum utama dari teks putusan.
    Ini adalah implementasi SANGAT SEDERHANA. Anda perlu mengembangkan ini.
    """
    if not isinstance(text, str) or not text:
        return ""

    match_menimbang_memutuskan = re.search(r'(?i)Menimbang\s+bahwa(.+?)(?:Memutuskan|MENGADILI|Menghukum|Mengadili)\s*(.+)', text, re.DOTALL)
    if match_menimbang_memutuskan:
        return match_menimbang_memutuskan.group(1).strip()

    match_memutuskan = re.search(r'(?i)Memutuskan(.+)', text, re.DOTALL)
    if match_memutuskan:
        return match_memutuskan.group(1).strip()

    return text[-300:].strip() + "..." if len(text) > 300 else text.strip()

def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    """
    Melakukan feature engineering sederhana pada DataFrame.
    Args:
        df (pd.DataFrame): DataFrame input.
    Returns:
        pd.DataFrame: DataFrame dengan fitur tambahan.
    """
    df['text_length'] = df['text_pdf'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)
    return df

print("Fungsi utilitas Tahap 2 berhasil didefinisikan.")

Fungsi utilitas Tahap 2 berhasil didefinisikan.


In [20]:
import io
import urllib
from pdfminer import high_level
import re

def get_pdf(url: str, path_pdf: str) -> Tuple[Union[io.BytesIO, None], str]:
    """
    Mengunduh file PDF dari URL dan mengembalikan stream konten serta nama file.
    Juga menyimpan PDF ke jalur yang ditentukan.
    """
    try:
        file = urllib.request.urlopen(url)
        file_name = file.info().get_filename()
        if file_name:
            file_name = re.sub(r'[\\/:*?"<>|]', '_', file_name)
        else:
            file_name = os.path.basename(url).split('?')[0]
            if not file_name.endswith('.pdf'):
                file_name += '.pdf'
            file_name = f"{os.path.splitext(file_name)[0]}_{int(time.time())}.pdf"

        file_content = file.read()
        full_pdf_path = os.path.join(path_pdf, file_name)
        with open(full_pdf_path, "wb") as out_file:
            out_file.write(file_content)
        print(f"PDF berhasil diunduh dan disimpan: {full_pdf_path}")
        return io.BytesIO(file_content), file_name
    except Exception as e:
        print(f"Gagal mengunduh atau memproses PDF dari {url}: {e}")
        return None, ""

def clean_text(text: str) -> str:
    """
    Membersihkan teks putusan dari header, footer, disclaimer, dan normalisasi spasi.
    """
    if not isinstance(text, str):
        return ""

    text = re.sub(r'M a h ka m a h A g u n g R e p u blik In d o n esia\n', '', text)
    text = re.sub(r'Disclaimer\n', '', text)
    text = re.sub(r'Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n', '', text)
    text = re.sub(r'pelaksanaan fungsi peradilan\. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu\.\n', '', text)
    text = re.sub(r'Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n', '', text)
    text = re.sub(r'Email : kepaniteraan@mahkamahagung\.go\.id\s+Telp : 021-384 3348 \(ext\.318\)\n', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [21]:
import pandas as pd
from datetime import date
from concurrent.futures import ThreadPoolExecutor, wait

def is_url_already_scraped(url: str, destination_csv_path: str) -> bool:
    """
    Memeriksa apakah URL sudah pernah di-scrape dan disimpan di file CSV.
    """
    if not os.path.isfile(f"{destination_csv_path}.csv"):
      return False

    try:
        df = pd.read_csv(f"{destination_csv_path}.csv")
        return url in df["link"].values
    except pd.errors.EmptyDataError:
        return False
    except Exception as e:
        print(f"Error saat memeriksa CSV yang sudah ada: {e}")
        return False

def extract_data(link: str, keyword_url: str, path_output_csv: str, path_pdf_files: str, path_raw_texts: str, download_pdf: bool = True):
    """
    Mengekstrak data detail dari halaman putusan, mengunduh PDF,
    membersihkan teks, dan menyimpannya ke CSV dan file teks.
    """
    global today
    today = date.today().strftime("%Y-%m-%d")

    keyword_for_filename = keyword_url.replace("/", "_").replace(":", "_").replace(".", "").replace("?", "").strip()
    if keyword_for_filename.startswith("https"):
        keyword_for_filename = "url_scrape_" + str(abs(hash(keyword_url)))[:10]
    destination_csv = os.path.join(path_output_csv, f"putusan_ma_{keyword_for_filename}_{today}")

    if is_url_already_scraped(link, destination_csv):
        print(f"Melewati URL duplikat: {link}")
        return

    soup = open_page(link)
    if soup is None:
        return

    table = soup.find("table", {"class": "table"})
    if table is None:
        print(f"Tidak dapat menemukan tabel detail putusan di {link}. Melewati.")
        return

    judul = table.find("h2").text if table.find("h2") else ""
    if table.find("h2"):
      table.find("h2").decompose()

    nomor = get_detail(table, "Nomor")
    tingkat_proses = get_detail(table, "Tingkat Proses")
    klasifikasi = get_detail(table, "Klasifikasi")
    kata_kunci = get_detail(table, "Kata Kunci")
    tahun = get_detail(table, "Tahun")
    tanggal_register = get_detail(table, "Tanggal Register")
    lembaga_peradilan = get_detail(table, "Lembaga Peradilan")
    jenis_lembaga_peradilan = get_detail(table, "Jenis Lembaga Peradilan")
    hakim_ketua = get_detail(table, "Hakim Ketua")
    hakim_anggota = get_detail(table, "Hakim Anggota")
    panitera = get_detail(table, "Panitera")
    amar = get_detail(table, "Amar")
    amar_lainnya = get_detail(table, "Amar Lainnya")
    catatan_amar = get_detail(table, "Catatan Amar")
    tanggal_musyawarah = get_detail(table, "Tanggal Musyawarah")
    tanggal_dibacakan = get_detail(table, "Tanggal Dibacakan")
    kaidah = get_detail(table, "Kaidah")
    abstrak = get_detail(table, "Abstrak")

    link_pdf = ""
    text_from_pdf = ""
    file_name_pdf = ""
    if download_pdf:
        try:
            link_pdf_tag = soup.find("a", href=re.compile(r"/pdf/"))
            if link_pdf_tag:
                link_pdf = link_pdf_tag["href"]
                if not link_pdf.startswith("http"):
                    link_pdf = f"https://putusan3.mahkamahagung.go.id{link_pdf}"

                file_pdf_stream, file_name_pdf = get_pdf(link_pdf, path_pdf_files)
                if file_pdf_stream:
                    text_from_pdf = high_level.extract_text(file_pdf_stream)
                    text_from_pdf = clean_text(text_from_pdf)

                    cleaned_nomor = re.sub(r'[^\w\s.-]', '', str(nomor)).replace(' ', '_') if pd.notna(nomor) else ""
                    case_id = cleaned_nomor if cleaned_nomor else f"case_{abs(hash(link))}"
                    raw_text_filename = os.path.join(path_raw_texts, f"{case_id}.txt")
                    with open(raw_text_filename, "w", encoding="utf-8") as f:
                        f.write(text_from_pdf)
                    print(f"Teks bersih disimpan di: {raw_text_filename}")
                else:
                    print(f"Tidak dapat mengekstrak teks dari PDF untuk {link}. Mungkin PDF kosong atau korup.")
            else:
                print(f"Link PDF tidak ditemukan untuk {link}. Melewati ekstraksi PDF.")
        except Exception as e:
            print(f"Error saat mengunduh/memproses PDF untuk {link}: {e}")
            link_pdf = ""
            text_from_pdf = ""
            file_name_pdf = ""

    data = [
        judul, nomor, tingkat_proses, klasifikasi, kata_kunci, tahun,
        tanggal_register, lembaga_peradilan, jenis_lembaga_peradilan,
        hakim_ketua, hakim_anggota, panitera, amar, amar_lainnya,
        catatan_amar, tanggal_musyawarah, tanggal_dibacakan, kaidah,
        abstrak, link, link_pdf, file_name_pdf, text_from_pdf,
    ]

    result = pd.DataFrame(
        [data],
        columns=[
            "judul", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun",
            "tanggal_register", "lembaga_peradilan", "jenis_lembaga_peradilan",
            "hakim_ketua", "hakim_anggota", "panitera", "amar", "amar_lainnya",
            "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah",
            "abstrak", "link", "link_pdf", "file_name_pdf", "text_pdf",
        ],
    )

    try:
        if not os.path.isfile(f"{destination_csv}.csv"):
            result.to_csv(f"{destination_csv}.csv", header=True, index=False)
            print(f"CSV baru dibuat dan data ditambahkan: {destination_csv}.csv")
        else:
            result.to_csv(f"{destination_csv}.csv", mode="a", header=False, index=False)
            print(f"Data ditambahkan ke CSV yang sudah ada: {destination_csv}.csv")
    except Exception as e:
        print(f"Error saat menyimpan data ke CSV {destination_csv}.csv: {e}")

In [22]:
# SEL 5: Proses Case Representation (Dengan Diagnostik Lebih Lanjut)

# Pastikan Anda telah menjalankan Sel 1 hingga Sel 4 sebelumnya.

# Definisikan ulang jalur agar bisa diakses di cell ini
base_drive_path = '/content/drive/MyDrive/CBR_Data'
processed_data_folder = os.path.join(base_drive_path, 'data/processed')
raw_texts_folder = os.path.join(base_drive_path, 'data/raw') # Folder ini berisi teks putusan bersih
pdf_files_folder = os.path.join(base_drive_path, 'PDF_Files_Raw') # Folder ini berisi PDF asli

# --- PENTING: Tentukan keyword_for_filename_from_tahap1 ---
# Berdasarkan output verifikasi Tahap 1 Anda sebelumnya, keyword/hash yang benar adalah:
# 'putusan_ma_url_scrape_3984702482_2025-06-27.csv'
keyword_for_filename_from_tahap1 = "url_scrape_3984702482" # <<< PASTIKAN INI SAMA DENGAN BAGIAN NAMA FILE DI DRIVE ANDA!

print(f"--- Memulai Proses Case Representation (Tahap 2) ---")
print(f"Jalur folder data Tahap 1: '{processed_data_folder}'")
print(f"Keyword/ID file Tahap 1 yang digunakan: '{keyword_for_filename_from_tahap1}'")

# --- LANGKAH 1: Memuat data dari Tahap 1 ---
print("\n[STEP 1/4] Memuat data CSV dari Tahap 1...")
df_cases = load_scraped_data(processed_data_folder, keyword_for_filename_from_tahap1)

# VERIFIKASI KRITIS: Periksa apakah df_cases kosong setelah pemuatan
if df_cases.empty:
    print("\n!!! DIAGNOSIS KRITIS: DataFrame 'df_cases' KOSONG setelah pemuatan data Tahap 1. Proses Tahap 2 TIDAK AKAN DILANJUTKAN.")
    print("   Penyebab paling mungkin: ")
    print(f"   1. Folder '{processed_data_folder}' TIDAK DAPAT DIAKSES oleh Colab saat ini (meskipun mungkin ada di Drive).")
    print(f"   2. File CSV 'putusan_ma_{keyword_for_filename_from_tahap1}_*.csv' TIDAK DITEMUKAN di folder tersebut, atau")
    print("   3. File CSV tersebut ditemukan tapi KOSONG atau RUSAK sehingga tidak bisa dibaca oleh Pandas.")
    print("   Mohon periksa kembali output dari fungsi 'load_scraped_data' di atas untuk detail lebih lanjut dan pastikan koneksi Drive stabil.")
    print("\n--- PROSES TAHAP 2 (CASE REPRESENTATION) SELESAI (Dengan Kegagalan Pemuatan Data) ---")

else: # Lanjutkan jika df_cases TIDAK KOSONG
    print(f"Sukses: DataFrame 'df_cases' dimuat dengan {len(df_cases)} baris.")
    print("Kolom yang ada di df_cases setelah pemuatan:")
    print(df_cases.columns.tolist())
    print("Sampel df_cases (head):")
    print(df_cases.head())

    # --- LANGKAH 2: Memastikan dan mengisi kolom 'text_pdf' ---
    print("\n[STEP 2/4] Memastikan dan mengisi kolom 'text_pdf'...")
    initial_text_pdf_count = df_cases['text_pdf'].count()
    if 'text_pdf' not in df_cases.columns or df_cases['text_pdf'].isnull().all() or (df_cases['text_pdf'] == '').all():
        print("Peringatan: Kolom 'text_pdf' kosong atau tidak ditemukan. Mencoba memuat teks dari file .txt di folder raw_texts_folder...")
        if 'text_pdf' not in df_cases.columns:
            df_cases['text_pdf'] = ''

        rows_to_fill = df_cases[df_cases['text_pdf'].isnull() | (df_cases['text_pdf'] == '')].shape[0]
        print(f"  Mencoba mengisi 'text_pdf' untuk {rows_to_fill} baris yang kosong/NaN.")

        for index, row in df_cases.iterrows():
            if pd.isna(row['text_pdf']) or row['text_pdf'] == '':
                nomor = row['nomor']
                link = row['link']

                cleaned_nomor = re.sub(r'[^\w\s.-]', '', str(nomor)).replace(' ', '_') if pd.notna(nomor) else ""
                case_id = cleaned_nomor if cleaned_nomor else f"case_{abs(hash(link))}"
                raw_text_filename = os.path.join(raw_texts_folder, f"{case_id}.txt")

                if os.path.exists(raw_text_filename):
                    try:
                        with open(raw_text_filename, 'r', encoding='utf-8') as f:
                            text_content = f.read()
                            df_cases.at[index, 'text_pdf'] = text_content
                        # print(f"  DEBUG: Teks dimuat untuk {case_id} dari {raw_text_filename} (Length: {len(text_content)} chars)") # uncomment for verbose debug
                    except Exception as e:
                        print(f"  ERROR: Gagal membaca teks dari {raw_text_filename}: {e}")
                # else:
                    # print(f"  DEBUG: File teks bersih tidak ditemukan untuk kasus: {case_id} di {raw_text_filename}. Melewati.")

    final_text_pdf_count = df_cases['text_pdf'].count()
    if final_text_pdf_count == 0:
        print("Peringatan Kritis: Kolom 'text_pdf' tetap kosong setelah semua upaya. Ekstraksi fakta/argumen mungkin tidak efektif.")
    else:
        print(f"Sukses: Kolom 'text_pdf' terisi (setelah pengisian). Total non-null: {final_text_pdf_count}")
        if final_text_pdf_count < len(df_cases):
            print(f"  ({len(df_cases) - final_text_pdf_count} baris 'text_pdf' masih kosong/NaN)")


    # --- LANGKAH 3: Ekstraksi Konten Kunci dan Feature Engineering ---
    print("\n[STEP 3/4] Melakukan Ekstraksi Konten Kunci dan Feature Engineering...")
    df_cases['ringkasan_fakta'] = df_cases['text_pdf'].apply(summarize_facts)
    df_cases['argumen_hukum_utama'] = df_cases['text_pdf'].apply(extract_legal_arguments)
    df_cases = feature_engineering(df_cases)
    print("Sukses: Kolom 'ringkasan_fakta', 'argumen_hukum_utama', 'text_length' ditambahkan.")
    print("Sampel kolom baru (head):")
    print(df_cases[['ringkasan_fakta', 'argumen_hukum_utama', 'text_length']].head())

    # --- LANGKAH 4: Persiapan dan Penyimpanan ke cases.csv ---
    print("\n[STEP 4/4] Mempersiapkan dan menyimpan 'cases.csv'...")
    columns_to_save = [
        "nomor", "tanggal_register", "judul", "tingkat_proses", "klasifikasi", "kata_kunci",
        "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota",
        "panitera", "amar", "amar_lainnya", "catatan_amar", "tanggal_musyawarah",
        "tanggal_dibacakan", "kaidah", "abstrak", "link", "link_pdf",
        "file_name_pdf", "text_pdf", # text_pdf disimpan juga untuk referensi
        "ringkasan_fakta", "argumen_hukum_utama", "text_length" # Fitur baru
    ]

    for col in columns_to_save:
        if col not in df_cases.columns:
            df_cases[col] = pd.NA

    df_cases_structured = df_cases[columns_to_save]

    # FINAL CHECK sebelum menyimpan: Pastikan df_cases_structured tidak kosong
    if df_cases_structured.empty:
        print("\n!!! DIAGNOSIS KRITIS: DataFrame 'df_cases_structured' KOSONG sebelum mencoba menyimpan. 'cases.csv' tidak akan dibuat.")
        print("   Ini mungkin karena df_cases kosong dari awal atau semua baris terfilter/terbuang.")
    else:
        output_cases_csv_path = os.path.join(processed_data_folder, 'cases.csv')
        print(f"Jalur penyimpanan untuk cases.csv: {output_cases_csv_path}")
        try:
            df_cases_structured.to_csv(output_cases_csv_path, index=False, encoding='utf-8')
            print(f"\nSukses: Data kasus terstruktur berhasil disimpan ke: {output_cases_csv_path}")
        except Exception as e:
            print(f"ERROR KRITIS: Gagal saat menyimpan cases.csv ke '{output_cases_csv_path}': {e}")
            print("Ini mungkin masalah izin tulis Google Drive, nama file yang bermasalah, atau koneksi.")

print("\n--- PROSES TAHAP 2 (CASE REPRESENTATION) SELESAI ---")

--- Memulai Proses Case Representation (Tahap 2) ---
Jalur folder data Tahap 1: '/content/drive/MyDrive/CBR_Data/data/processed'
Keyword/ID file Tahap 1 yang digunakan: 'url_scrape_3984702482'

[STEP 1/4] Memuat data CSV dari Tahap 1...

--- Memulai load_scraped_data ---
Jalur folder yang diperiksa: /content/drive/MyDrive/CBR_Data/data/processed
Keyword/Hint yang diberikan: 'url_scrape_3984702482'
File-file yang ditemukan di '/content/drive/MyDrive/CBR_Data/data/processed': ['putusan_ma_url_scrape_3984702482_2025-06-27.csv']
DIAGNOSIS: Ditemukan file CSV spesifik hari ini: putusan_ma_url_scrape_3984702482_2025-06-27.csv
DIAGNOSIS: Akan mencoba memuat file: /content/drive/MyDrive/CBR_Data/data/processed/putusan_ma_url_scrape_3984702482_2025-06-27.csv
Berhasil memuat 65 dokumen dari /content/drive/MyDrive/CBR_Data/data/processed/putusan_ma_url_scrape_3984702482_2025-06-27.csv
Sukses: DataFrame 'df_cases' dimuat dengan 65 baris.
Kolom yang ada di df_cases setelah pemuatan:
['judul', 'nomo

In [23]:
# SEL BARU UNTUK VERIFIKASI HASIL TAHAP 2

print("Memverifikasi file cases.csv di Google Drive...")

base_drive_path = '/content/drive/MyDrive/CBR_Data'
processed_data_folder = os.path.join(base_drive_path, 'data/processed')
output_cases_csv_path = os.path.join(processed_data_folder, 'cases.csv')

print(f"\n--- Memeriksa '{output_cases_csv_path}': ---")
if os.path.exists(output_cases_csv_path):
    print(f"File '{output_cases_csv_path}' ditemukan.")
    try:
        df_verify = pd.read_csv(output_cases_csv_path)
        print(f"Berhasil memuat {len(df_verify)} baris dari cases.csv.")
        print("\nLima baris pertama dari cases.csv:")
        print(df_verify.head())
        print("\nKolom-kolom yang ada di cases.csv:")
        print(df_verify.columns.tolist())
    except Exception as e:
        print(f"Error saat membaca cases.csv: {e}")
else:
    print(f"File '{output_cases_csv_path}' TIDAK DITEMUKAN. Ada masalah saat penyimpanan atau jalur salah.")

print("\nVerifikasi Tahap 2 selesai.")
print("Jika file terlihat di sini tetapi tidak di Google Drive web, coba refresh Google Drive web Anda.")

Memverifikasi file cases.csv di Google Drive...

--- Memeriksa '/content/drive/MyDrive/CBR_Data/data/processed/cases.csv': ---
File '/content/drive/MyDrive/CBR_Data/data/processed/cases.csv' ditemukan.
Berhasil memuat 65 baris dari cases.csv.

Lima baris pertama dari cases.csv:
                      nomor  tanggal_register  \
0   805/Pid.Sus/2016/PN TBT                 —   
1    1489/Pdt.G/2025/PA.Pwd       3 Juni 2025   
2  1985/Pid.Sus/2016/PN Plg                 —   
3   174/PID.SUS/2013/PN.BDG   6 Februari 2013   
4   478/Pid.Sus/2016/PN Bgl  29 Nopember 2016   

                                               judul tingkat_proses  \
0  Putusan PN TEBING TINGGI Nomor 805/Pid.Sus/201...        Pertama   
1  Putusan PA PURWODADI Nomor 1489/Pdt.G/2025/PA....        Pertama   
2  Putusan PN PALEMBANG Nomor 1985/Pid.Sus/2016/P...        Pertama   
3  Putusan PN BANDUNG Nomor 174/PID.SUS/2013/PN.B...        Pertama   
4  Putusan PN BENGKULU Nomor 478/Pid.Sus/2016/PN ...        Pertama   
